In [1]:
'''
Notebook for the finding a_b scaling factor for simulating it with variable cell volume
'''
# By Kirill Sechkar

# PACKAGE IMPORTS 
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS
import synthetic_circuits as circuits
from cell_model import *

cpu


Loading BokehJS ...

In [2]:
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit - WITH VARIABLE-VOLUME SIMULATION SUPPORT
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het, \
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, circuit_v = cellmodel_auxil.add_circuit(
    circuits.punisher_cnc_b_initialise,
    circuits.punisher_cnc_b_ode,
    circuits.punisher_cnc_b_F_calc,
    circuits.punisher_cnc_b_eff_m_het_div_k_het,
    par, init_conds,
    # propensity calculation function for variable-volume simulations
    circuits.punisher_cnc_b_v_varvol, varvol=True)

In [3]:
# PARAMETERISE THE CIRCUIT

# BURDENSOME SYNTHETIC GENE
par['c_b'] = 1
par['a_b'] = 1e5

# PUNISHER
# switch gene conc
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch'] = 0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch'] / 80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0  # 0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
par['a_cat'] = 500.0  # promoter strength (unitless)
par['n_cat'] = 300.0
# synthetic protease gene
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0  # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2  # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# plasmid copy number control
init_conds['cat_pb'] = 10.0  # INITIAL CONDITION (not a parameter): all plasmids have working CAT gene copies
par['k_tr'] = 130.2  # plasmid replication rate (1/h)
par['a_inh'] = 948  # inhibitor synthesis rate per plasmid copy (1/h)
par['b_inh'] = 74.976  # inhibitor degradation rate (1/h)
par['n_inh'] = 10  # number of steps of replication initiation at which inhibition can happen
par['K_inh'] = 214.05  # replication inhibition constant (nM)

# critical cell volume triggering division
par['V_crit'] = 2.0 * np.log(2)  # 2ln(2) so as to have an average volume of 1 um^3 assuming constant growth rate

# BURDENSOME GENE REPLICATION
par['mean_rep_phase_b'] = 0.5  # mean replication phase
par['stdev_rep_phase_b'] = 0.23  # standard deviation of replication phase (void as considering avergae dynamics for now)
# scaling factor, makes average burdensome gene concentration equal to that determined by the fixed-volume model
# a_b_scale = (par['V_crit']/2) * np.log(2) * np.exp(par['mean_rep_phase_b']*np.log(2))

# culture medium
nutr_qual = 0.5
par['s'] = nutr_qual  # nutrient quality (unitless)
init_conds['s'] = nutr_qual  # nutrient quality (unitless)
par['h_ext']=10.5*(10**3)

In [4]:
# DEFINE THE RANGE OF VALUES TO CONSIDER
a_b_scales = np.linspace(0.65,0.9,6)
last_cycles_considered = 10  # number of last full cell cycles to inlude when calculating the average p_b

In [5]:
# FIXED-VOLUME SIMULATION TO FIND THE STARTING STEADY STATE - SET SIMULATION PARAMETERS
# set simulation parameters
tf = (0, 50)  # simulation time frame
savetimestep = 0.1  # save time step
dt_max = 0.1  # maximum integration step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

In [6]:
# VARIABLE-VOLUME SIMULATION WITH VARIABLE CELL VOLUME - SET SIMULATION PARAMETERS
tf_varvol = (tf[-1], tf[-1] + 10.0)  # simulation time frame with variable volume
tau = 1e-7  # simulation time step
tau_odestep = 1e-7  # number of ODE integration steps in a single tau-leap step (smaller than tau)
tau_savetimestep = 1e-2  # save time step a multiple of tau

In [7]:
# FIXED-VOLUME SIMULATION TO FIND THE STARTING STEADY STATE - RUN

sol = ode_sim(par,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts_det = np.array(sol.ts)
xs_det = np.array(sol.ys)
# det_steady_x = jnp.concatenate((sol.ys[-1, 0:8], jnp.round(sol.ys[-1, 8:])))
det_steady_x = sol.ys[-1, :]

# record the fixed-volume burdensome protein concentration
p_b_fixedvol =xs_det[-1, circuit_name2pos['p_b']]

In [8]:
# VARIABLE-VOLUME SIMULATION WITH VARIABLE CELL VOLUME - RUN
p_bs=np.zeros_like(a_b_scales)  # initialise the burdensome protein concentrations
a_b_default=par['a_b']
for i in range(len(a_b_scales)):
    mRNA_count_scales, S, x0_tauleap, circuit_synpos2genename, keys0, \
        rep_phase_means_stdevs_bounds = tauleap_sim_prep_varvol(par, len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                                                    det_steady_x,
                                                    key_seeds=[0]
                                                    )
    x0_tauleap[6]=1.0 # start at the default volume of 1 um^3
    par['a_b']=a_b_default*a_b_scales[i] # apply the scaling factor to the burdensome gene transcription rate
    ts_jnp_varvol, xs_jnp_varvol, final_keys_varvol = tauleap_sim_varvol(par,  # dictionary with model parameters
                                             circuit_v,  # circuit reaction propensity calculator
                                             circuit_eff_m_het_div_k_het,
                                             x0_tauleap,     # initial condition VECTOR (processed to make sure random variables are appropriate integers)
                                             len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                                             cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes), # synthetic gene parameters for calculating k values
                                             tf_varvol, tau, tau_odestep, tau_savetimestep,    # simulation parameters: time frame, tau leap step size, number of ode integration steps in a single tau leap step
                                             mRNA_count_scales, S, circuit_synpos2genename, # mRNA count scaling factor, stoichiometry matrix, synthetic gene number in list of synth. genes to name decoder
                                             keys0, rep_phase_means_stdevs_bounds,
                                             avg_dynamics=True)  # starting random number genereation key
    
    # concatenate the results with the deterministic simulation
    ts = np.concatenate((ts_det, np.array(ts_jnp_varvol)))
    xs_first = np.concatenate((xs_det, np.array(xs_jnp_varvol[0])))  # getting the results from the first random number generator key in vmap
    xss_varvol = np.concatenate((xs_det * np.ones((keys0.shape[0], 1, 1)), np.array(xs_jnp_varvol)),axis=1)  # getting the results from all vmapped trajectories
    
    # get molecule concentrations
    Vs = xs_first[:, 6]  # cell volumes
    xs_concs = np.divide(xs_first, (Vs * np.ones_like(np.array([xs_first[0,:]]).T)).T)  # divide abundances by cell volumes to get concentrations
    xs_concs[:, 6] = par['s'] * np.ones_like(Vs)  # instead of volumes, x without variable volumes has nutrient quality in this position
    
    # consider the last x cell cycles
    div_time_indices=np.where(Vs[1:] < Vs[:-1])[0]+1  # find division times
    print(div_time_indices)
    last_cycles_start_index=div_time_indices[-(last_cycles_considered+1)]  # find the start of the last several full cell cycles considered
    last_cycles_end_index=div_time_indices[-1]  # find the end of the last several full cell cycles considered
    p_bs[i]=np.mean(xs_concs[last_cycles_start_index:last_cycles_end_index, circuit_name2pos['p_b']])  # calculate the average burdensome protein concentration

[525 575]
[525 577]


In [9]:
# FIND THE SCALING CORRESPONDING TO CLOSEST AVERAGE BURDENSOME PROTEIN CONCENTRATION
print('Fixed-volume burdensome protein concentration:'+str(p_b_fixedvol))  # print the fixed-volume burdensome protein concentration
print('\n')
print('Scaling factors:'+str(a_b_scales))  # print the scaling factors
print('Average burdensome protein concentrations:'+str(p_bs))  # print the average burdensome protein concentrations

best_scale_index=np.argmin(np.abs(p_bs-p_b_fixedvol))  # find the scaling factor that gives the closest average burdensome protein concentration
best_scale=a_b_scales[best_scale_index]  # find the best scaling factor

print('Best scaling factor:'+str(best_scale))  # print the best scaling factor
print('Stands for the average of '+str(p_bs[best_scale_index]))

Fixed-volume burdensome protein concentration:204692.32516385138


Scaling factors:[0.5, 1]
Average burdensome protein concentrations:[179631.85684773 222507.45484483]
Best scaling factor:1
Stands for the average of 222507.45484482733
